In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import os


def aes_generate_key():
    # 256bit random data
    return os.urandom(32)


def aes_gcm_encrypt(key: bytes, message: bytes) -> bytes:
    # Generate 96bit Initialization Vector
    iv = os.urandom(12)

    # Init AES-GCM
    cipher = Cipher(algorithms.AES(key), modes.GCM(iv))
    encryptor = cipher.encryptor()
    # Encrypt 'message'
    aes_data = encryptor.update(message) + encryptor.finalize()
    # Get the full cipherText: IV + Encrypted-Message + Auth-Tag
    ciphertext = iv + aes_data + encryptor.tag
    return ciphertext


def aes_gcm_decrypt(key: bytes, ciphertext: bytes) -> bytes:
    iv = ciphertext[0:12]
    aes_data = ciphertext[12:-16]
    tag = ciphertext[-16:]

    cipher = Cipher(algorithms.AES(key), modes.GCM(iv, tag))
    decryptor = cipher.decryptor()
    text = decryptor.update(aes_data) + decryptor.finalize()
    return text


if __name__ == '__main__':
    MESSAGE = b'hello world'

    aes_key = aes_generate_key()
    ciphertext = aes_gcm_encrypt(aes_key, MESSAGE)
    plaintext = aes_gcm_decrypt(aes_key, ciphertext)
    assert plaintext == MESSAGE, (plaintext, MESSAGE)

    print('AES key', base64.b64encode(aes_key))
    print('Ciphertext', base64.b64encode(ciphertext))


AES key b'dMWxxh1KqCSLRhU93XPuj5md8LWWtqLTREO8Z45KMbs='
Ciphertext b'M/zUOygaL2LEdkZvFH7LtNWSc8Yw03Mnu0GcUBLuyuRW8xxhaUiK'


In [2]:
#!/usr/bin/env python3
# ----------------------------------------------------------------
# Copyright (c) 2023 Matteo Bertozzi
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.
#

# pip3 install cryptography
from cryptography.hazmat.primitives.serialization import *
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import hashes

import base64
import os


def generate_rsa_keys(key_size: int = 4096) -> tuple[rsa.RSAPrivateKey, rsa.RSAPublicKey]:
    private_key = rsa.generate_private_key(
        public_exponent=65537, key_size=key_size)
    public_key = private_key.public_key()
    return private_key, public_key


def export_rsa_public_key(public_key: rsa.RSAPublicKey) -> bytes:
    public_key_spki = public_key.public_bytes(
        encoding=Encoding.DER,
        format=PublicFormat.SubjectPublicKeyInfo
    )
    return public_key_spki


def export_rsa_private_key(private_key: rsa.RSAPrivateKey) -> bytes:
    private_key_pkcs8 = private_key.private_bytes(
        encoding=Encoding.DER,
        format=PrivateFormat.PKCS8,
        encryption_algorithm=NoEncryption()  # Optional encryption algorithm
    )
    return private_key_pkcs8


def load_rsa_public_key(public_key_spki: bytes) -> rsa.RSAPublicKey:
    return load_der_public_key(public_key_spki)


def load_rsa_private_key(private_key_pkcs8: bytes) -> rsa.RSAPrivateKey:
    return load_der_private_key(private_key_pkcs8, password=None)


OAEP_PADDING = padding.OAEP(
    mgf=padding.MGF1(algorithm=hashes.SHA512()),
    algorithm=hashes.SHA512(),
    label=None
)


def rsa_oaep_encrypt(public_key: rsa.RSAPublicKey, message: bytes) -> bytes:
    return public_key.encrypt(message, OAEP_PADDING)


def rsa_oaep_decrypt(private_key: rsa.RSAPrivateKey, ciphertext: bytes) -> bytes:
    return private_key.decrypt(ciphertext, OAEP_PADDING)


def rsa_sign_sha256(private_key: rsa.RSAPrivateKey, message: bytes) -> bytes:
    return private_key.sign(
        message,
        padding.PKCS1v15(),
        hashes.SHA256()
    )


def rsa_verify_sign_sha256(public_key: rsa.RSAPublicKey, signature: bytes, message: bytes) -> bytes:
    public_key.verify(
        signature,
        message,
        padding.PKCS1v15(),
        hashes.SHA256()
    )


if __name__ == '__main__':
    MESSAGE = b'hello world'

    # Generate RSA key pair
    private_key, public_key = generate_rsa_keys()

    # Encrypt and decrypt using RSA-OAEP
    ciphertext = rsa_oaep_encrypt(public_key, MESSAGE)
    plaintext = rsa_oaep_decrypt(private_key, ciphertext)
    
    # Ensure decryption is successful
    assert plaintext == MESSAGE, (plaintext, MESSAGE)

    # Sign the message and verify the signature
    signature = rsa_sign_sha256(private_key, MESSAGE)
    rsa_verify_sign_sha256(public_key, signature, MESSAGE)

    # Print results
    print('RSA public-key:', base64.b64encode(export_rsa_public_key(public_key)).decode('utf-8'))
    print('RSA private-key:', base64.b64encode(export_rsa_private_key(private_key)).decode('utf-8'))
    print('Ciphertext (Length: {}):'.format(len(ciphertext)), base64.b64encode(ciphertext).decode('utf-8'))
    print('Signature (Length: {}):'.format(len(signature)), base64.b64encode(signature).decode('utf-8'))


RSA public-key: MIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEAp6WxIq11i1CfZb64RCj+C6w3U8qP0xsLH3Pq9kfDC47FqvqcU47JqqWsPQmPLRqgp71dpS5jk6SOSUAREzZUJVYEbeznV5D5q0qY3TEFF1+FLIEGegdcRGGcF3ZGzw7utFz1h9xwbh4jYVPjmWjxvNL/fWqPk8CIguFy6UNZm8XrpJm27zR+J4zJhkareJei13hOiljqvFmZ6j+fL0sop5lBnDfwfYlHo0F+Fb9v4e7MSXd+Kd0N0686fUrhLHQ2sZU5YgfTVWUMP2WyCA64zPNoX3NsG74fhllG1S798aVwnEfoX80t/bGKjr+KZm1XTMpSbHQjwRD9n7dfzHextzCf6Pn3hSxIsM5L2VDgGKF6YAueBefidyURugjWf5PscEZnsMV9Rr5qtni/YWsnDICnYp+hgqFsKerbBr3w8Y3OqspCzuDYRKl9DjXs+7AtNGwrVoHpjRpphnu70Cr11LH5LxecQqL78E+W6HzYyt174jV8WV4aJ6iUJ7aaZRWWBu/851lYSVgm4skyeZx93IXrRolYOpYeN31hOYGKxUEI6OBNBFxbioVx+/C6IB4PpQw5/y4LosCO6Li5S+OWpSyJHdkbNJtf28fwGVKeKn9h+sb6Y0JZUHji8SKIGOF0lL5/qb8VEYezfWcu4rS0XBi3mvtAWMl6qic5U/myqbMCAwEAAQ==
RSA private-key: MIIJQgIBADANBgkqhkiG9w0BAQEFAASCCSwwggkoAgEAAoICAQCnpbEirXWLUJ9lvrhEKP4LrDdTyo/TGwsfc+r2R8MLjsWq+pxTjsmqpaw9CY8tGqCnvV2lLmOTpI5JQBETNlQlVgRt7OdXkPmrSpjdMQUXX4UsgQZ6B1xEYZwXdkbPDu60XPWH3HBuHiNhU+OZaPG80v99ao+TwIiC4XLpQ1mbxeukmbbvNH